In [1]:
from utils.libs import *

In [2]:
TARGET_DATE = '2020.12.13-232457'
UNIFORM_CSV_DIR = 'uniform_csv_files'
KIND_FREQ = "high-freq"
ARR_WTS_NO = np.array([128,3161,2795,2795,2246,2246,64])
RAW_CSV_PATH = os.path.join(f"C:\\Users\\Francesco\\Desktop\\processed-results-qat", f"{KIND_FREQ}-linear_quant_attempt_{TARGET_DATE}.csv")

FLAG_JOIN_TO_CMPRS_DF_CSV = False
FLAG_SAVE_TO_DF_CSV = False

In [3]:
N_HF, N_HL = 64, 5
W, H = 256, 256
baseline_values_ws = "128,4096,4096,4096,4096,4096,64".split(",")
baseline_values_ws = list(map(float, baseline_values_ws))

tot_ws_baseline = sum(baseline_values_ws)
print('Tot number weigths', tot_ws_baseline)

baseline_values_bias = "2,64,64,64,64,64,1".split(",")
baseline_values_bias = list(map(float, baseline_values_bias))

tot_bias_baseline = sum(baseline_values_bias)
print('Tot number biases', tot_bias_baseline)

BASELINE_MODEL_SIZE_BIT = (tot_bias_baseline + tot_ws_baseline) * 32
print("Baseline(Byte)", BASELINE_MODEL_SIZE_BIT / 8, "Baseline(bit)", BASELINE_MODEL_SIZE_BIT )

Tot number weigths 20672.0
Tot number biases 323.0
Baseline(Byte) 83980.0 Baseline(bit) 671840.0


In [4]:
raw_df = pd.read_csv(RAW_CSV_PATH)
raw_df.head(5)

,date,MSE,PSNR,SSIM,TIME,bits_activations,bits_bias,bits_weights,class,ema_decay,...,module.net.3.linear.weight_zero_point,module.net.4.linear.float_weight,module.net.4.linear.weight,module.net.4.linear.weight_scale,module.net.4.linear.weight_zero_point,module.net.5.linear.float_weight,module.net.5.linear.weight,module.net.5.linear.weight_scale,module.net.5.linear.weight_zero_point,module.net.6.weight
0,___2021.01.12-013206,0.002306,32.393727,0.887593,0.007880,32,32,8,QuantAwareTrainRangeLinearQuantizer,0.999,...,64.0,2048,2036,64.0,64.0,2048,2041,64.0,64.0,64
1,___2021.01.12-014045,0.000435,39.641515,0.971622,0.008161,32,32,8,QuantAwareTrainRangeLinearQuantizer,0.999,...,64.0,2048,2048,64.0,64.0,2048,2046,64.0,64.0,64
2,___2021.01.12-014916,0.002730,31.659620,0.882123,0.007745,32,32,8,QuantAwareTrainRangeLinearQuantizer,0.999,...,NaN,2048,2032,NaN,NaN,2048,2014,NaN,NaN,64
3,___2021.01.12-015731,0.000632,38.018303,0.961999,0.007856,32,32,8,QuantAwareTrainRangeLinearQuantizer,0.999,...,NaN,2048,2048,NaN,NaN,2048,2046,NaN,NaN,64
4,___2021.01.12-020550,0.003730,30.303459,0.870930,0.008169,32,32,8,QuantAwareTrainRangeLinearQuantizer,0.999,...,64.0,2048,2034,64.0,64.0,2048,2016,64.0,64.0,64


In [5]:
raw_df.columns

Index(['date', 'MSE', 'PSNR', 'SSIM', 'TIME', 'bits_activations', 'bits_bias',
       'bits_weights', 'class', 'ema_decay', 'mode', 'overrides',
       'per_channel_wts', 'frequency', 'quantizer', 'starting_epoch',
       'module.net.0.linear.weight', 'module.net.1.linear.float_weight',
       'module.net.1.linear.weight', 'module.net.1.linear.weight_scale',
       'module.net.1.linear.weight_zero_point',
       'module.net.2.linear.float_weight', 'module.net.2.linear.weight',
       'module.net.2.linear.weight_scale',
       'module.net.2.linear.weight_zero_point',
       'module.net.3.linear.float_weight', 'module.net.3.linear.weight',
       'module.net.3.linear.weight_scale',
       'module.net.3.linear.weight_zero_point',
       'module.net.4.linear.float_weight', 'module.net.4.linear.weight',
       'module.net.4.linear.weight_scale',
       'module.net.4.linear.weight_zero_point',
       'module.net.5.linear.float_weight', 'module.net.5.linear.weight',
       'module.net.5.linea

In [6]:
columns = list(raw_df.columns)
def calculate_model_size(a_row, columns=columns, hl=N_HL, hf=N_HF, arr_wts_no = ARR_WTS_NO):
    a_row_dict = dict(zip(columns, a_row))
    wts = [f"module.net.{ii}.linear.weight" for ii in range(0, 5+1)]
    wts_list = dict(filter(lambda item: item[0] in wts, a_row_dict.items()))
    a_row_dict[f"module.net.{hl+1}.weight"]
    wts_list[f"module.net.{hl+1}.weight"] = a_row_dict[f"module.net.{hl+1}.weight"]
    
    overrides = eval(a_row_dict["overrides"])
    arr_bits_weights = []
    arr_wts_no = []
    for k, v in overrides.items():
        bits_weights = v['bits_weights'] if v['bits_weights'] else 32
        arr_bits_weights.append(bits_weights)
        wts_no = wts_list[f"module.{k}.weight"]
        arr_wts_no.append(wts_no)
    
    arr_bits_weights = np.array(arr_bits_weights) # ;pprint(arr_bits_weights)
    # pprint(arr_bits_weights * arr_wts_no)
    model_size_byte = (32 * 2 + np.sum(arr_bits_weights * arr_wts_no) + 32 * hl * hf + 32) / 8
    # print("Model size byte:", model_size_byte)
    return model_size_byte

raw_df['model_size_byte'] = list(map(calculate_model_size, raw_df.values[0:]))

In [7]:
raw_df['bpp'] = list(map(lambda item: item * 8 / (W * H), raw_df["model_size_byte"].values))
raw_df['CR'] = list(map(lambda item: (W*H*8) / (item * 8), raw_df["model_size_byte"].values))

In [8]:
columns_picked = "date,MSE,PSNR,SSIM,TIME,class,mode,per_channel_wts,model_size_byte,CR,bpp".split(",")
raw_uniform_df = copy.deepcopy(raw_df[columns_picked])
raw_uniform_df.head(5)

,date,MSE,PSNR,SSIM,TIME,class,mode,per_channel_wts,model_size_byte,CR,bpp
0,___2021.01.12-013206,0.002306,32.393727,0.887593,0.007880,QuantAwareTrainRangeLinearQuantizer,SYMMETRIC,True,14476.0,4.527217,1.767090
1,___2021.01.12-014045,0.000435,39.641515,0.971622,0.008161,QuantAwareTrainRangeLinearQuantizer,SYMMETRIC,True,14547.0,4.505121,1.775757
2,___2021.01.12-014916,0.002730,31.659620,0.882123,0.007745,QuantAwareTrainRangeLinearQuantizer,SYMMETRIC,False,14420.0,4.544799,1.760254
3,___2021.01.12-015731,0.000632,38.018303,0.961999,0.007856,QuantAwareTrainRangeLinearQuantizer,SYMMETRIC,False,14548.0,4.504812,1.775879
4,___2021.01.12-020550,0.003730,30.303459,0.870930,0.008169,QuantAwareTrainRangeLinearQuantizer,SYMMETRIC,True,14465.0,4.530660,1.765747


In [9]:
rename_columns = "MSE,PSNR,SSIM,TIME,model_size_byte".split(",")
renamed_columns = "mse,psnr,ssim,time,size(byte)".split(",")
for old_col_name, new_col_name in zip(rename_columns, renamed_columns):
    if old_col_name not in raw_uniform_df.columns: continue
    raw_uniform_df[f"{new_col_name}"] = raw_uniform_df[f"{old_col_name}"].values
    raw_uniform_df = raw_uniform_df.drop([f"{old_col_name}"], axis = 1)
    pass

raw_uniform_df["date"] = list(map(lambda item: item[3:], raw_uniform_df["date"].values))
raw_uniform_df["footprint(%)"] = raw_uniform_df["size(byte)"] * 8 / BASELINE_MODEL_SIZE_BIT * 100

def create_compression_class_column(item, prune_tech = 'AGP'):
    class_attr, mode_attr, per_channel_wts_attr = item
    class_name, mode_name, pcw_name = None, None, 'NNPCW'
    if per_channel_wts_attr:
        pcw_name = 'PCW'
    if len(mode_attr.split("_")) > 1:
        mode_name = ''.join(list(map(lambda item: item[0].upper(), mode_attr.split("_"))))
    else:
        mode_name = mode_attr[0].upper()
    class_name = ''.join(list(filter(lambda letter: letter != letter.lower(), list(class_attr))))
    if prune_tech:
        return f"{prune_tech}+{class_name}:{mode_name}:{pcw_name}"
    return f"{class_name}:{mode_name}:{pcw_name}"

drop_columns = "class,mode,per_channel_wts".split(",")
raw_uniform_df["cmprss-class"] = list(map(create_compression_class_column, raw_uniform_df[drop_columns].values))
for old_col_name in drop_columns:
    if old_col_name not in raw_uniform_df.columns: continue
    raw_uniform_df = raw_uniform_df.drop([f"{old_col_name}"], axis = 1)
    pass

raw_uniform_df['init-from'] = [f'{TARGET_DATE}'] * raw_uniform_df.shape[0]
columns_order = "date,init-from,size(byte),footprint(%),psnr,bpp,CR,mse,ssim,cmprss-class".split(",")
raw_uniform_df = raw_uniform_df[columns_order]

raw_uniform_df.head(5)

,date,init-from,size(byte),footprint(%),psnr,bpp,CR,mse,ssim,cmprss-class
0,2021.01.12-013206,2020.12.13-232457,14476.0,17.237437,32.393727,1.767090,4.527217,0.002306,0.887593,AGP+QATRLQ:S:PCW
1,2021.01.12-014045,2020.12.13-232457,14547.0,17.321981,39.641515,1.775757,4.505121,0.000435,0.971622,AGP+QATRLQ:S:PCW
2,2021.01.12-014916,2020.12.13-232457,14420.0,17.170755,31.659620,1.760254,4.544799,0.002730,0.882123,AGP+QATRLQ:S:NNPCW
3,2021.01.12-015731,2020.12.13-232457,14548.0,17.323172,38.018303,1.775879,4.504812,0.000632,0.961999,AGP+QATRLQ:S:NNPCW
4,2021.01.12-020550,2020.12.13-232457,14465.0,17.224339,30.303459,1.765747,4.530660,0.003730,0.870930,AGP+QATRLQ:S:PCW


In [10]:
def create_compression_class_2(cmprss_class):
    if cmprss_class.startswith("jpeg".upper()):
        return cmprss_class.split(":")[0]
    if cmprss_class.startswith("agp".upper()):
        if len(cmprss_class.split("+")) == 1:
            return cmprss_class.split(":")[0]
    if cmprss_class.startswith("QATRLQ".upper()):
        pass
    if "QATRLQ".upper() in cmprss_class:
        return "QATRLQ"
    if cmprss_class.lower().startswith("Baseline".lower()):
        return "BASELINE" 
    return cmprss_class
raw_uniform_df["cmprss-class-2"] = list(map(create_compression_class_2, raw_uniform_df[f"cmprss-class"].values))
raw_uniform_df.head(5)

,date,init-from,size(byte),footprint(%),psnr,bpp,CR,mse,ssim,cmprss-class,cmprss-class-2
0,2021.01.12-013206,2020.12.13-232457,14476.0,17.237437,32.393727,1.767090,4.527217,0.002306,0.887593,AGP+QATRLQ:S:PCW,QATRLQ
1,2021.01.12-014045,2020.12.13-232457,14547.0,17.321981,39.641515,1.775757,4.505121,0.000435,0.971622,AGP+QATRLQ:S:PCW,QATRLQ
2,2021.01.12-014916,2020.12.13-232457,14420.0,17.170755,31.659620,1.760254,4.544799,0.002730,0.882123,AGP+QATRLQ:S:NNPCW,QATRLQ
3,2021.01.12-015731,2020.12.13-232457,14548.0,17.323172,38.018303,1.775879,4.504812,0.000632,0.961999,AGP+QATRLQ:S:NNPCW,QATRLQ
4,2021.01.12-020550,2020.12.13-232457,14465.0,17.224339,30.303459,1.765747,4.530660,0.003730,0.870930,AGP+QATRLQ:S:PCW,QATRLQ


In [11]:
if FLAG_SAVE_TO_DF_CSV:
    raw_uniform_csv_path = os.path.join(f"{UNIFORM_CSV_DIR}", f"{KIND_FREQ}-siren_qatrlq_uniform_{TARGET_DATE}.csv")
    print("Saving raw data to structured csv in:")
    print(f"\t{raw_uniform_csv_path}")
    raw_uniform_df.to_csv(raw_uniform_csv_path)

In [12]:
if FLAG_JOIN_TO_CMPRS_DF_CSV:
    cmprs_df_csv_path = os.path.join(f"{UNIFORM_CSV_DIR}", "cmprs_df.csv")   
    print("Appending structured data in:")
    print(f"\t{cmprs_df_csv_path}")

    cmprs_df = pd.read_csv(cmprs_df_csv_path)
    if "Unnamed: 0" in cmprs_df.columns:
        cmprs_df = cmprs_df.drop(["Unnamed: 0"], axis = 1)
    
    print('Before:')
    print('CMPRS DF:', cmprs_df.shape)
    print('RAW DF:', raw_uniform_df.shape)
    
    list_dfs = [cmprs_df, raw_uniform_df]
    cmprs_df = pd.concat(list_dfs, axis = 0, ignore_index=True)
    cmprs_df_csv_path = os.path.join(f"{UNIFORM_CSV_DIR}", "cmprs_df.csv")
    cmprs_df.to_csv(cmprs_df_csv_path)
    
    print('After:')
    print(cmprs_df.shape)
    print(cmprs_df.head(5))
    pass